In [1]:
# https://www.kaggle.com/kernels/scriptcontent/11511967/notebook

import string
import numpy as np
import pandas as pd
# from IPython.display import display
from tqdm import tqdm
from collections import Counter
import ast
import re
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
# import matplotlib.mlab as mlab
import seaborn as sb

from sklearn.feature_extraction.text import CountVectorizer
# from textblob import TextBlob
import scipy.stats as stats

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE

import nltk
from nltk.tag import tnt
from nltk.corpus import stopwords
from nltk.corpus import indian
from nltk.tokenize import word_tokenize,sent_tokenize

import gensim
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import gensim.corpora as corpora
from gensim.summarization import summarize

import stanfordnlp

import warnings
warnings.filterwarnings("ignore")

# from bokeh.plotting import figure, output_file, show
# from bokeh.models import Label
# from bokeh.io import output_notebook
# output_notebook()

%matplotlib inline

/Users/rohanraj/opt/anaconda3/lib/python3.7/site-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable
/Users/rohanraj/opt/anaconda3/lib/python3.7/site-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping
/Users/rohanraj/opt/anaconda3/lib/python3.7/site-packages/nltk/decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
/Users/rohanraj/opt/anaconda3/lib/python3.7/site-packages/nltk/lm/counter.py:15: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, an

In [2]:
nlp = stanfordnlp.Pipeline ( lang = 'hi' ) 

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/Users/rohanraj/stanfordnlp_resources/hi_hdtb_models/hi_hdtb_tokenizer.pt', 'lang': 'hi', 'shorthand': 'hi_hdtb', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/Users/rohanraj/stanfordnlp_resources/hi_hdtb_models/hi_hdtb_tagger.pt', 'pretrain_path': '/Users/rohanraj/stanfordnlp_resources/hi_hdtb_models/hi_hdtb.pretrain.pt', 'lang': 'hi', 'shorthand': 'hi_hdtb', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/Users/rohanraj/stanfordnlp_resources/hi_hdtb_models/hi_hdtb_lemmatizer.pt', 'lang': 'hi', 'shorthand': 'hi_hdtb', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': '/Users/rohanraj/stanfordnlp_resources/hi_hdtb_models/hi_hdtb_parser.pt', 'pretrain_path': '/Users/rohanraj/stanford

In [3]:
datafile = '../data/hindi.txt'
with open(datafile,'r',encoding='utf-8') as f:
    text = f.read()
    text = text.split("॥")

In [88]:
text_sentence = []
i = 0
for word in tqdm(text):
#     print(i,word)
    if re.findall("^\s?\d+\s?",word):
        continue
    else:   
        text_sentence.append(word)

100%|██████████| 1290/1290 [00:00<00:00, 423401.84it/s]


In [90]:
text_sentence[0:20]

[' वानर हनुमान सीमा के देखने की उत्कण्ठा येसूयास्त की प्रतीक्षा करता भया ',
 ' घ४-९) सीता को रावण के भन्तापुर में ढूंढना : वह महावीर्य महान हृदय पाछा वानर श्रेष्ठ रात के समय अद्वार से कोट को फांद कर का में प्रविष्ट हृया ',
 'धानरराज के उस हितैषी ने लंका नगरी में प्रवेश करके मानों अपना बायां पाओं शत्रु के सिर पर रख दिया ',
 " उस समय सुन्दर 'सब ओर से सजे हुए श्वेत मेघ के तुल्य राक्षसों के जो पद्माकार, स्वस्तिकाकार, और वर्धमान घर है, उन से इंका जगमन. कर रही थी, राधा के अर्थ वह श्रीमान् घूमता हुमा उसे देखता भया और आनन्दित होता भया ।। ३, ४ ",
 ' एक भवन से दूसरे भवन को जाते हुए उस वानरश्रेष्ठ ने वहाँ र विविध भाकृति और रूपों वाले भवन देखे ',
 ' वहाँ राक्षसों के घरों में उसने अप "करते हुओं के मन्त्र मुने और स्वाध्याय में रत राक्षसों को देखा ',
 ' राक्षसों के घर से घर और बगीचों को देखता हुआ बेधड़क यह महलों के पास घूमा ',
 ' तव पवनपुत्र हनुमान ने यह भवन श्रेष्ठ देखा, जो राक्षसपति का भवन है, और बहुत महलों से भरपूर है',
 ' विशाल नेत्रोंवाली वैदेही सीता को ढूंढता हुआ शत्रुनों के मारनेवाला, हनुमान उ

In [6]:
len(text_sentence) #total sentences

667

In [7]:
# hindi stopwords
stop_words_df = pd.read_csv("../data/stopwords.txt", header = None)
stop_words = list(set(stop_words_df.values.reshape(1,-1).tolist()[0]))
stop_words.extend(["।", "।।", ")", "(", ",",'"',"हे", "हो", 'में','से','COMMA'])

In [8]:
len(stop_words)

723

In [9]:
stop_words[:10]

['कौनसे',
 'जाएंगी',
 'दे',
 'कराएं',
 'लेने',
 'सुनी',
 'दोनो',
 'किया',
 'देने',
 'में']

In [10]:
# Define helper functions
def get_top_n_words(n_top_words, text_data, stop_words):
    '''
    returns a tuple of the top n words in a sample and their 
    accompanying counts, given a CountVectorizer object and text sample
    '''
    count_vectorizer = CountVectorizer(stop_words=stop_words) 
    vectorized_headlines = count_vectorizer.fit_transform(text_data)
#     print(vectorized_headlines)
    vectorized_total = np.sum(vectorized_headlines, axis=0)
    word_indices = np.flip(np.argsort(vectorized_total)[0,:], 1)
    word_values = np.flip(np.sort(vectorized_total)[0,:],1)
    
    word_vectors = np.zeros((n_top_words, vectorized_headlines.shape[1]))
    for i in range(n_top_words):
        word_vectors[i,word_indices[0,i]] = 1

    words = [word[0] for 
             word in count_vectorizer.inverse_transform(word_vectors)]

    return (words, word_values[0,:n_top_words].tolist()[0])

In [11]:
words, word_values = get_top_n_words(n_top_words=5,text_data=text_sentence, stop_words= stop_words)

In [12]:
words, word_values

(['मह', 'सम', 'नर', 'नह', 'षस'], [110, 108, 104, 102, 100])

In [132]:
#from nltk import FreqDist

In [133]:
#fdist1 = FreqDist(str_text[0:2500])

In [134]:
#fdist1

In [13]:
def create_str_from_list(original_read_text):
    prepared_text = ""
    for line in original_read_text:
        line = line.split()
#       print(line)
        tmp_line = " ".join(line)
        prepared_text += " \n"+tmp_line
    return prepared_text

In [14]:
str_text = create_str_from_list(text_sentence)

In [106]:
text_pos = nlp(str_text[0:2500])

In [127]:
text_pos.sentences[0].tokens[0]

<Token index=1;words=[<Word index=1;text=वानर;lemma=वानर;upos=NOUN;xpos=NN;feats=Case=Nom|Gender=Masc|Number=Sing|Person=3;governor=3;dependency_relation=nmod>]>

In [131]:
text_pos.sentences[0].words[0].lemma

'वानर'

In [123]:
text_pos.sentences[0].tokens_string()

"<Token index=1;words=[<Word index=1;text=वानर;lemma=वानर;upos=NOUN;xpos=NN;feats=Case=Nom|Gender=Masc|Number=Sing|Person=3;governor=3;dependency_relation=nmod>]>\n<Token index=2;words=[<Word index=2;text=हनुमान;lemma=हनुमान;upos=PROPN;xpos=NNPC;feats=Case=Nom|Gender=Masc|Number=Sing|Person=3;governor=3;dependency_relation=compound>]>\n<Token index=3;words=[<Word index=3;text=सीमा;lemma=सीमा;upos=PROPN;xpos=NNP;feats=Case=Acc|Gender=Fem|Number=Sing|Person=3;governor=5;dependency_relation=nmod>]>\n<Token index=4;words=[<Word index=4;text=के;lemma=का;upos=ADP;xpos=PSP;feats=AdpType=Post;governor=3;dependency_relation=case>]>\n<Token index=5;words=[<Word index=5;text=देखने;lemma=देख;upos=VERB;xpos=VM;feats=Case=Acc|Number=Sing|VerbForm=Inf;governor=8;dependency_relation=nmod>]>\n<Token index=6;words=[<Word index=6;text=की;lemma=का;upos=ADP;xpos=PSP;feats=AdpType=Post|Case=Acc|Gender=Fem|Number=Sing;governor=5;dependency_relation=mark>]>\n<Token index=7;words=[<Word index=7;text=उत्कण्ठा;l

In [24]:
str_text[0:2500]

' \nवानर हनुमान सीमा के देखने की उत्कण्ठा येसूयास्त की प्रतीक्षा करता भया \nघ४-९) सीता को रावण के भन्तापुर में ढूंढना : वह महावीर्य महान हृदय पाछा वानर श्रेष्ठ रात के समय अद्वार से कोट को फांद कर का में प्रविष्ट हृया \nधानरराज के उस हितैषी ने लंका नगरी में प्रवेश करके मानों अपना बायां पाओं शत्रु के सिर पर रख दिया \nउस समय सुन्दर \'सब ओर से सजे हुए श्वेत मेघ के तुल्य राक्षसों के जो पद्माकार, स्वस्तिकाकार, और वर्धमान घर है, उन से इंका जगमन. कर रही थी, राधा के अर्थ वह श्रीमान् घूमता हुमा उसे देखता भया और आनन्दित होता भया ।। ३, ४ \nएक भवन से दूसरे भवन को जाते हुए उस वानरश्रेष्ठ ने वहाँ र विविध भाकृति और रूपों वाले भवन देखे \nवहाँ राक्षसों के घरों में उसने अप "करते हुओं के मन्त्र मुने और स्वाध्याय में रत राक्षसों को देखा \nराक्षसों के घर से घर और बगीचों को देखता हुआ बेधड़क यह महलों के पास घूमा \nतव पवनपुत्र हनुमान ने यह भवन श्रेष्ठ देखा, जो राक्षसपति का भवन है, और बहुत महलों से भरपूर है \nविशाल नेत्रोंवाली वैदेही सीता को ढूंढता हुआ शत्रुनों के मारनेवाला, हनुमान उसके चारों ओर घूमा ।। ९ \nतब 

In [59]:
summarize(str_text[0:2500])

"घ४-९) सीता को रावण के भन्तापुर में ढूंढना : वह महावीर्य महान हृदय पाछा वानर श्रेष्ठ रात के समय अद्वार से कोट को फांद कर का में प्रविष्ट हृया \nउस समय सुन्दर 'सब ओर से सजे हुए श्वेत मेघ के तुल्य राक्षसों के जो पद्माकार, स्वस्तिकाकार, और वर्धमान घर है, उन से इंका जगमन.\nराक्षसों के घर से घर और बगीचों को देखता हुआ बेधड़क यह महलों के पास घूमा \nतब वह उस सुन्दर पड़ी माला की ओर मस्थित हुआ, जो उत्तम स्त्री की तरह रावण की बड़ी पारी थी \nमें बस वानर यूयपति ने चन्द्र तुल्य मुखवाली, सुन्दर कुण्डल पहने हुई, ताजे घुषषों की पालाएं और भूषणोंवाली पनियों को देसा "

In [26]:
#https://github.com/USC-NLP-17/hindiTextExtraction

import networkx as nx
import numpy as np
import math
import re

from nltk.tokenize.punkt import PunktSentenceTokenizer
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

In [135]:
def textrankTfIdf(document):
    # sentence_tokenizer = PunktSentenceTokenizer()
    # sentences = sentence_tokenizer.tokenize(document, 'hindi')

    sentences = document
    bow_matrix = CountVectorizer().fit_transform(sentences)
    # normalized = TfidfTransformer(norm='l2', use_idf=True, use_bm25idf=True, smooth_idf=True,
    #              delta_idf=False, sublinear_tf=False, bm25_tf=True).fit_transform(bow_matrix)

    normalized = TfidfTransformer().fit_transform(bow_matrix)
    similarity_graph = normalized * normalized.T

    nx_graph = nx.from_scipy_sparse_matrix(similarity_graph)
    scores = nx.pagerank(nx_graph)
    return sorted(((scores[i], s) for i, s in enumerate(sentences)),
                  reverse=True)

In [162]:
def orderSentences(rankedList, data, initSentences):
    index = ['']*len(data)
    # print(rankedList)
    for eachRanked in rankedList[0:int(math.ceil(0.05*len(rankedList)))]:
        sen = eachRanked[1]
        index[data.index(sen)] = initSentences[data.index(sen)]
        # print(data.index(sen))
    return index

In [163]:
rankedSentences = textrankTfIdf(text_sentence[0:25])
orderedsentences = orderSentences(rankedSentences, text_sentence[0:25], text_sentence[0:25])

In [164]:
orderedsentences

['',
 '',
 '',
 " उस समय सुन्दर 'सब ओर से सजे हुए श्वेत मेघ के तुल्य राक्षसों के जो पद्माकार, स्वस्तिकाकार, और वर्धमान घर है, उन से इंका जगमन. कर रही थी, राधा के अर्थ वह श्रीमान् घूमता हुमा उसे देखता भया और आनन्दित होता भया ।। ३, ४ ",
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ' जो अपनी शोभा से मानों उस उत्तम भवन को शोभायमान करती थी, बद्द मन्दोदरी थी, जोकि सुन्दर रूपवती वहां लेटी हुई थी ',
 '',
 '',
 '',
 '',
 '',
 '']

In [165]:
for ordered in orderedsentences:
    if ordered != "":
        print(ordered)
        #print(rankedSentencesDoc.write(str(ordered) + u'\u0964' + "\n"))

 उस समय सुन्दर 'सब ओर से सजे हुए श्वेत मेघ के तुल्य राक्षसों के जो पद्माकार, स्वस्तिकाकार, और वर्धमान घर है, उन से इंका जगमन. कर रही थी, राधा के अर्थ वह श्रीमान् घूमता हुमा उसे देखता भया और आनन्दित होता भया ।। ३, ४ 
 जो अपनी शोभा से मानों उस उत्तम भवन को शोभायमान करती थी, बद्द मन्दोदरी थी, जोकि सुन्दर रूपवती वहां लेटी हुई थी 


In [30]:
for ordered in orderedsentences:
    if ordered != "":
        print(ordered)
        #print(rankedSentencesDoc.write(str(ordered) + u'\u0964' + "\n"))

 उस समय सुन्दर 'सब ओर से सजे हुए श्वेत मेघ के तुल्य राक्षसों के जो पद्माकार, स्वस्तिकाकार, और वर्धमान घर है, उन से इंका जगमन. कर रही थी, राधा के अर्थ वह श्रीमान् घूमता हुमा उसे देखता भया और आनन्दित होता भया ।। ३, ४ 
 एक भवन से दूसरे भवन को जाते हुए उस वानरश्रेष्ठ ने वहाँ र विविध भाकृति और रूपों वाले भवन देखे 
 वहाँ राक्षसों के घरों में उसने अप "करते हुओं के मन्त्र मुने और स्वाध्याय में रत राक्षसों को देखा 
 तब वह उस सुन्दर पड़ी माला की ओर मस्थित हुआ, जो उत्तम स्त्री की तरह रावण की बड़ी पारी थी 
 ओर शराब पीकर बन्द हुए, श्रीद चमकते हुए पलङ्ग -. पर लेटे हुए राक्षसाधिपति को उस महारानर ने देखा 
 जो अपनी शोभा से मानों उस उत्तम भवन को शोभायमान करती थी, बद्द मन्दोदरी थी, जोकि सुन्दर रूपवती वहां लेटी हुई थी 
 य०१०-११) राधण के अन्तःपुर में सीता का न पाना पर उसी समय उस रूपाळ को पटाकर, स्थित हुा महा . घानर सीता के विषय भरा विचार करता भया 
 नि:सन्देह यह कोई और है, ऐसा निश्चय करके फिर वह विचरने लगा, " इसमकार रावण का सारा अन्तःपुर( रनिवास ) उस महातेजस्वी वानर ने पूरी तरह देखा, पर जानकी को नहीं देखा 


In [31]:
text_sentence[0:25]

[' वानर हनुमान सीमा के देखने की उत्कण्ठा येसूयास्त की प्रतीक्षा करता भया ',
 ' घ४-९) सीता को रावण के भन्तापुर में ढूंढना : वह महावीर्य महान हृदय पाछा वानर श्रेष्ठ रात के समय अद्वार से कोट को फांद कर का में प्रविष्ट हृया ',
 'धानरराज के उस हितैषी ने लंका नगरी में प्रवेश करके मानों अपना बायां पाओं शत्रु के सिर पर रख दिया ',
 " उस समय सुन्दर 'सब ओर से सजे हुए श्वेत मेघ के तुल्य राक्षसों के जो पद्माकार, स्वस्तिकाकार, और वर्धमान घर है, उन से इंका जगमन. कर रही थी, राधा के अर्थ वह श्रीमान् घूमता हुमा उसे देखता भया और आनन्दित होता भया ।। ३, ४ ",
 ' एक भवन से दूसरे भवन को जाते हुए उस वानरश्रेष्ठ ने वहाँ र विविध भाकृति और रूपों वाले भवन देखे ',
 ' वहाँ राक्षसों के घरों में उसने अप "करते हुओं के मन्त्र मुने और स्वाध्याय में रत राक्षसों को देखा ',
 ' राक्षसों के घर से घर और बगीचों को देखता हुआ बेधड़क यह महलों के पास घूमा ',
 ' तव पवनपुत्र हनुमान ने यह भवन श्रेष्ठ देखा, जो राक्षसपति का भवन है, और बहुत महलों से भरपूर है',
 ' विशाल नेत्रोंवाली वैदेही सीता को ढूंढता हुआ शत्रुनों के मारनेवाला, हनुमान उ

In [36]:
def lemmatization(text):
    lemmatized_text = []
    for line in tqdm(text):
        if line not in [""," "] :
            doc = nlp(line)
            for sent in doc.sentences:
                for wrd in sent.words:
                    #extract text and lemma
                    lemmatized_text.append(wrd.lemma)
    return lemmatized_text

In [37]:
lemmatized = lemmatization(text_sentence[0:25])

100%|██████████| 25/25 [00:02<00:00, 10.28it/s]


In [126]:
lemmatized[0:7]

['वानर', 'हनुमान', 'सीमा', 'का', 'देख', 'का', 'उत्कण्ठा']

In [39]:
def remove_stopwords(word_tokenized,stop_words):
    return [word for word in word_tokenized if word not in stop_words]

In [40]:
clean_text = remove_stopwords(lemmatized,stop_words)
len(clean_text),len(lemmatized)

(281, 612)

In [42]:
clean_text[0:5]

['वानर', 'हनुमान', 'सीमा', 'उत्कण्ठा', 'येसूयास्त']

In [47]:
# remove punctuation from each word
table = str.maketrans('', '', string.punctuation)
stripped = [w.translate(table) for w in clean_text]
print(stripped[:25])
print(len(clean_text),len(stripped))

['वानर', 'हनुमान', 'सीमा', 'उत्कण्ठा', 'येसूयास्त', 'प्रतीक्षा', 'भया', 'घ४९', 'सीता', 'रावण', 'भन्तापुर', 'ढूँढ', '', 'महावीर्य', 'महान', 'हृदय', 'पाछ', 'वानर', 'श्रेष्ठ', 'रात', 'अद्वार', 'कोट', 'फांद', 'प्रविष्ट', 'हृ']
281 281


In [48]:
def custom_remove_garbage(original_words_list,list_of_garbage_words):
    tmp_list = [word for word in original_words_list if word not in list_of_garbage_words] # garbage list
    tmp_list = [word for word in tmp_list if len(re.findall("\d+",word))==0] # english numbers
    tmp_list = [word for word in tmp_list if len(re.findall("[a-zA-Z]+",word))==0] # english alphabets
    return tmp_list

In [49]:
clean_text == stripped

# Python code to get difference of two lists 
# Not using set() 
def Diff(li1, li2): 
    li_dif = [i for i in li1 + li2 if i not in li1 or i not in li2] 
    return li_dif 

resultant = Diff(clean_text,stripped)
len(set(resultant))

19

In [50]:
resultant

['घ४-९',
 ':',
 'SINGLE_QUOTE',
 'जगमन.',
 '-.',
 'पीछे-इट',
 'घर.',
 'य०१०-११',
 '.',
 'नि:सन्देह',
 'अन्तःपुर(',
 'घ४९',
 '',
 'SINGLEQUOTE',
 'जगमन',
 '',
 'पीछेइट',
 'घर',
 'य०१०११',
 '',
 'निसन्देह']

In [51]:
final_text = custom_remove_garbage(stripped,resultant)
len(final_text)

265

In [52]:
final_text[0:5]

['वानर', 'हनुमान', 'सीमा', 'उत्कण्ठा', 'येसूयास्त']

In [43]:
# Create the Dictionary and Corpus needed for Topic Modeling
# The two main inputs to the LDA topic model are the dictionary(id2word) and the corpus.

# Create Dictionary
id2word = corpora.Dictionary([clean_text])

# Create Corpus
texts = clean_text

# Term Document Frequency
corpus = [id2word.doc2bow(texts)] #for text in texts]

# Gensim creates a unique id for each word in the document. The produced corpus shown above is a mapping of 
# (word_id, word_frequency).
# For example, (0, 1) above implies, word id 0 occurs once in the first document. Likewise, word id 1 occurs twice and so on.

# View
# print(corpus[:1])
print(id2word[1]) # word a given id corresponds to

.


In [44]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('-.', 1),
  ('.', 1),
  (':', 1),
  ('SINGLE_QUOTE', 1),
  ('अत्यन्त', 2),
  ('अद्वार', 1),
  ('अधिपति', 1),
  ('अन्तःपुर', 1),
  ('अन्तःपुर(', 1),
  ('अप', 1),
  ('अर्थ', 1),
  ('अलङ्कार', 1),
  ('आनन्दित', 1),
  ('इंका', 1),
  ('इनुमान्', 1),
  ('इसमकार', 1),
  ('इा', 1),
  ('उत्कण्ठा', 1),
  ('उत्तम', 2),
  ('उत्पन्न', 1),
  ('उम', 1),
  ('उमक', 1),
  ('ऊँचा', 1),
  ('एकान्त', 1),
  ('एला', 1),
  ('कंचा', 1),
  ('कदाचित', 1),
  ('कान्ति', 1),
  ('काम', 1),
  ('कुण्डल', 1),
  ('कुलीन', 1),
  ('कोट', 1),
  ('खिय', 1),
  ('खो', 1),
  ('गलीचा', 1),
  ('गीवन', 1),
  ('गुभ', 1),
  ('घपलोप', 1),
  ('घर.', 1),
  ('घानर', 1),
  ('घुषष', 1),
  ('घूम', 1),
  ('घूमा', 2),
  ('घ४-९', 1),
  ('चन्द्र', 1),
  ('चमक', 2),
  ('चार', 1),
  ('छौ', 1),
  ('जगमन.', 1),
  ('जड़', 1),
  ('जल', 1),
  ('जानकी', 1),
  ('जिमकी', 1),
  ('जोकि', 1),
  ('झरोक', 1),
  ('डर', 1),
  ('ढूँढ', 1),
  ('ढूँढता', 1),
  ('तव', 1),
  ('ताजा', 1),
  ('तुल्य', 2),
  ('तेज', 1),
  ('दान्त', 1),
  ('दिव्य', 1),
  ('दीपक', 1

In [45]:
# Build LDA model

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3, 
                                           random_state=100,
                                           update_every=10,
                                           chunksize=100,
                                           passes=20,
                                           alpha='auto',
                                           per_word_topics=True)

In [46]:
# Print the Keyword in the 10 topics
# It means the top 10 keywords that contribute to this topic are: ‘hockey’, ‘pts_pt’, ‘pit’.. and 
# so on and the weight of ‘hockey’ on topic 0 is 0.044.

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.018*"भवन" + 0.015*"सीता" + 0.012*"सुन्दर" + 0.012*"रावण" + 0.012*"राक्षस" '
  '+ 0.012*"भया" + 0.012*"वानर" + 0.009*"स्त्री" + 0.009*"मा" + 0.009*"सक्ती"'),
 (1,
  '0.004*"भवन" + 0.004*"सीता" + 0.004*"वानर" + 0.004*"भया" + 0.004*"रावण" + '
  '0.004*"राक्षस" + 0.004*"मा" + 0.004*"मान" + 0.004*"सुन्दर" + 0.004*"सक्ती"'),
 (2,
  '0.004*"भवन" + 0.004*"वानर" + 0.004*"भया" + 0.004*"हनुमान" + 0.004*"सीता" + '
  '0.004*"सुन्दर" + 0.004*"स्त्री" + 0.004*"राक्षस" + 0.004*"रावण" + '
  '0.004*"मान"')]


In [53]:
# Create the Dictionary and Corpus needed for Topic Modeling
# The two main inputs to the LDA topic model are the dictionary(id2word) and the corpus.

# Create Dictionary
id2word = corpora.Dictionary([final_text])

# Create Corpus
texts = final_text

# Term Document Frequency
corpus = [id2word.doc2bow(texts)] #for text in texts]

# Gensim creates a unique id for each word in the document. The produced corpus shown above is a mapping of 
# (word_id, word_frequency).
# For example, (0, 1) above implies, word id 0 occurs once in the first document. Likewise, word id 1 occurs twice and so on.

# View
# print(corpus[:1])
print(id2word[1]) # word a given id corresponds to

अद्वार


In [54]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('अत्यन्त', 2),
  ('अद्वार', 1),
  ('अधिपति', 1),
  ('अन्तःपुर', 2),
  ('अप', 1),
  ('अर्थ', 1),
  ('अलङ्कार', 1),
  ('आनन्दित', 1),
  ('इंका', 1),
  ('इनुमान्', 1),
  ('इसमकार', 1),
  ('इा', 1),
  ('उत्कण्ठा', 1),
  ('उत्तम', 2),
  ('उत्पन्न', 1),
  ('उम', 1),
  ('उमक', 1),
  ('ऊँचा', 1),
  ('एकान्त', 1),
  ('एला', 1),
  ('कंचा', 1),
  ('कदाचित', 1),
  ('कान्ति', 1),
  ('काम', 1),
  ('कुण्डल', 1),
  ('कुलीन', 1),
  ('कोट', 1),
  ('खिय', 1),
  ('खो', 1),
  ('गलीचा', 1),
  ('गीवन', 1),
  ('गुभ', 1),
  ('घपलोप', 1),
  ('घानर', 1),
  ('घुषष', 1),
  ('घूम', 1),
  ('घूमा', 2),
  ('चन्द्र', 1),
  ('चमक', 2),
  ('चार', 1),
  ('छौ', 1),
  ('जड़', 1),
  ('जल', 1),
  ('जानकी', 1),
  ('जिमकी', 1),
  ('जोकि', 1),
  ('झरोक', 1),
  ('डर', 1),
  ('ढूँढ', 1),
  ('ढूँढता', 1),
  ('तव', 1),
  ('ताजा', 1),
  ('तुल्य', 2),
  ('तेज', 1),
  ('दान्त', 1),
  ('दिव्य', 1),
  ('दीपक', 1),
  ('देसा', 1),
  ('धानरराज', 1),
  ('नगरी', 1),
  ('निश्चय', 1),
  ('नेत्रोंवाली', 1),
  ('पटा', 1),
  ('पड', 1),
  ('पद्म

In [57]:
# Build LDA model

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=10,
                                           chunksize=100,
                                           passes=20,
                                           alpha='auto',
                                           per_word_topics=True)

In [58]:
# Print the Keyword in the 10 topics
# It means the top 10 keywords that contribute to this topic are: ‘hockey’, ‘pts_pt’, ‘pit’.. and 
# so on and the weight of ‘hockey’ on topic 0 is 0.044.

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.005*"भवन" + 0.005*"राक्षस" + 0.005*"सीता" + 0.005*"मा" + 0.005*"हनुमान" + '
  '0.005*"वानर" + 0.005*"सुन्दर" + 0.005*"रावण" + 0.005*"भया" + 0.005*"रत"'),
 (1,
  '0.005*"सीता" + 0.005*"भवन" + 0.005*"सक्ती" + 0.005*"रावण" + 0.005*"भया" + '
  '0.005*"सुन्दर" + 0.005*"वानर" + 0.005*"राक्षस" + 0.005*"मान" + '
  '0.005*"हनुमान"'),
 (2,
  '0.005*"भवन" + 0.005*"सीता" + 0.005*"सुन्दर" + 0.005*"रावण" + 0.005*"स्त्री" '
  '+ 0.005*"वानर" + 0.005*"राक्षस" + 0.005*"मान" + 0.005*"भया" + '
  '0.005*"हनुमान"'),
 (3,
  '0.020*"भवन" + 0.017*"सीता" + 0.014*"भया" + 0.014*"रावण" + 0.014*"वानर" + '
  '0.014*"राक्षस" + 0.014*"सुन्दर" + 0.010*"सक्ती" + 0.010*"स्त्री" + '
  '0.010*"मान"'),
 (4,
  '0.005*"भवन" + 0.005*"भया" + 0.005*"सीता" + 0.005*"सुन्दर" + 0.005*"स्त्री" '
  '+ 0.005*"वानर" + 0.005*"मान" + 0.005*"राक्षस" + 0.005*"रावण" + '
  '0.005*"सक्ती"')]
